Merge data


In [3]:

import pandas as pd
import glob
import os
import json

# Đường dẫn tới thư mục chứa các file json đã làm sạch
folder_path = 'Clean_reviews'

# Lấy danh sách tất cả các file json trong thư mục
file_list = glob.glob(os.path.join(folder_path, '*.json'))
dfs = []
for file in file_list:
    # Đọc dữ liệu từ file json
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    # Thêm cột product_category dựa trên tên file
    filename = os.path.basename(file)
    # Lấy phần giữa 'tiki_reviews_' và '_cleaned.json'
    # Lấy category từ tên file, ví dụ: tiki_reviews_electronics_cleaned.json -> electronics
    parts = filename.replace('.json', '').split('_')
    if len(parts) >= 3:
        category = '_'.join(parts[2:-1])  # lấy tất cả phần giữa 'tiki_reviews_' và '_cleaned'
    else:
        category = 'unknown'
    df['product_category'] = category

    # Thay thế các trường images rỗng ([]) thành [""] trong DataFrame
    if 'images' in df.columns:
        df['images'] = df['images'].apply(lambda x: [""] if isinstance(x, list) and len(x) == 0 else x)

    dfs.append(df)

if len(dfs) == 0:
    raise ValueError("Không tìm thấy file JSON nào trong thư mục hoặc dữ liệu rỗng!")

# Gộp tất cả các dataframe lại thành một dataframe lớn
big_df = pd.concat(dfs, ignore_index=True)

# Đảm bảo thư mục tồn tại trước khi lưu file
os.makedirs('Data Processing/Data/Silver Data', exist_ok=True)
# Lưu ra file mới
big_df.to_json('all_reviews_with_category.json', orient='records', force_ascii=False)


Split

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

# Đọc dữ liệu từ file JSON đã lưu
df = pd.read_json('all_reviews_with_category.json', orient='records')   

# Chia 15% (train+test) và 85% (unsupervised)
df_supervised, df_unsupervised = train_test_split(df, test_size=0.85, random_state=42, shuffle=True)

# Tạo thư mục Manual_label_data nếu chưa có
os.makedirs('Manual_label_data', exist_ok=True)

# df_supervised là 15%, df_unsupervised là 85%
# Lưu manual.json vào Manual_label_data
df_supervised.to_json('Manual_label_data/manual.json', orient='records', force_ascii=False)
df_unsupervised.to_json('unsupervised.json', orient='records', force_ascii=False)

# Chia manual thành 80% train và 20% test
manual_df = pd.read_json('Manual_label_data/manual.json', orient='records')
train_df, test_df = train_test_split(manual_df, test_size=0.2, random_state=42, shuffle=True)

# Lưu train.json và test.json vào Manual_label_data
train_df.to_json('Manual_label_data/train.json', orient='records', force_ascii=False)
test_df.to_json('Manual_label_data/test.json', orient='records', force_ascii=False)

# Copy 210 sample từ test vào guideline_sample, chia làm 3 file
guideline_samples = test_df.sample(n=210, random_state=42)
os.makedirs('guideline_sample', exist_ok=True)
for i in range(3):
    start = i * 70
    end = (i + 1) * 70
    part = guideline_samples.iloc[start:end]
    part.to_json(f'guideline_sample/guideline_sample_{i+1}.json', orient='records', force_ascii=False)